## Intro

### TL;DR
You can use the sql database or two column dataframe produced in this book to select training and validation sets that are distributed similarly to the test set. [Dataset here](https://www.kaggle.com/calebeverett/riiid-folds).


### Motivation
In general, we want to train on data that is distributed as similarly as possible to the test data with respect to both the variable we are predicting and the features we are using in the model. We also want a validation set that is simiar to the test set, but doesn't have any leakage from the training set.

For this dataset, I thought is was important to ensure that the train and validation sets had distributions similar to the test set for the following variables:
* Percent of interactions answered correctly
* Number of interactions per user
* Where in users' histories interactions begin

The test set has been described as being comprised of mostly records that continue sequentially from records in the training data. If that is the case, then the training and validation sets should be constructed similarly, so that the included sequences of users' interactions start later in users' histories, in fact, towards the end of their histories based on the description of the test set. This seems especially important given the nature of users' learning and differences in the likely changes in the characteristics of the questions delivered to users' over time. For example, on the one hand, we would expect users to correctly answer a greater percentage of questions as their knowledge increases through the use of the system over time. On the other hand, though, users are delivered more difficult questions over time to advance their learning, which may otherwise may make it more difficult for them to answer questions correctly. Below, I compared users' performance on questions later in their histories with their earlier histories and found that there was a significant increase in performance over time. Regardless of the magnitude of these two illustative effects, the important point is that the characteristics of the later interactions are liklely different from the early ones and that since we are going to be tesing on the later ones, we should construct training and validation sets that are comprised mostly of later records.

I had initially thought that it was beneficial to train on records of users that would be seen in test set, but have since come to the conclusion that for decision tree based models, where there is no explicit relationship between records, and user_id isn't used as a feature, that the benefit is minimal. As long as we are able to construct the features our models use to make predictions from an accurate representation of each users' state at the beginning of the predicton period, it won't make much of a difference whether we are predicting on interactions of users that were in our training set. (Is this right?) As it is currently, if you train a model on a subset of the training data, it is likely that you are predicting on a test set that includes records primarily related to users you didn't train on. Perhaps there is something inherent in users' interactions that carries across the sequence of their interactions that is valuable, but it seems like as long as you have the state data for each user as of the start of the sequences in the test set, it is much more imporant to make sure that the time frames of the sequences are similar. 

### Methodology
The rest of this notebook constructs folds that can be used interchangeably for training or validation purposes. It divides the training data into 40 folds with approximately 1.2 million records in each that are distributed in accordance with the description provided by the competition organizers of the test set and mutually exclusive with respect to users. The end product is an sqlite database that can be easily read into pandas dataframes that includes the training data with a fold column appended. I've also created a two column (row_id, fold) pandas dataframe that can be joined with your data to select data sets.

* Users divided randomly into 40 folds
* Within each fold:
    * 10% of users have all of their records included (~20% of records)
    * 60% of users have only their records starting in the later half of their complete histories included (~30% of records) 
    * 20% of users have only their records starting in the in the first half of their complete histories included (~50% of records)
       
### Results
The objective of having good data sets is obviously to have better model results. I was looking for an improvement in model performance and for my validation scores to be closer to my leader board scores. While not an exhaustive analysis of the impact, I did train two models with the same features and hyperparameters on a similar number of records using my old split methodology and using this new methodology for the purpose of comparison. There was less of a difference here than I was expecting, but my validation score is now closer to my leaderboard score. It is difficult to draw any definitive conclusions from just two models, but I believe this metodology will ultimately lead to better results throughout the competition.

<table>
    <tr>
        <th>Train Dataset</th>
        <th>Valid Dataset</th>
        <th>Train AUC</th>
        <th>Valid AUC</th>
        <th>Public<br>Leaderboard</th>                    
    </tr>
    <tr>
        <td>
            <ul>
                <li>24.5 million records</li>
                <li>85k users</li>
                <li>100% of users (100% of records) from first interaction</li>
            </ul>
        </td>
        <td>
            <ul>
                <li>4.9 million records</li>
                <li>17k users</li>
                <li>10% of users (20% of records) from first interaction</li>
                <li>90% of users (80% of records) from beginning on average 50%
                    of the way through sequence of interactions</li>
            </ul>
        </td>
        <td>0.771</td>
        <td>0.764</td>
        <td>0.772</td>
    </tr>
        <tr>
        <td>
            <ul>
                <li>23.4 million records</li>
                <li>172k users</li>
                <li>10% of users (~20% of records) from first interaction</li>
                <li>60% of users (~30% of records) from an average of 75% of the way through sequence of interactions</li>
                <li>30% of users (~50% of records) from an average of 25% of the way through sequence of interactions</li>
            </ul>
        </td>
        <td>
            <ul>
                <li>2.6 million records</li>
                <li>19k users</li>
                <li>10% of users (~20% of records) from first interaction</li>
                <li>60% of users (~30% of records) from an average of 75% of the way through sequence of interactions</li>
                <li>30% of users (~50% of records) from an average of 25% of the way through sequence of interactions</li>
            </ul>
        </td>
        <td>0.773</td>
        <td>0.769</td>
        <td>0.773</td>
    </tr>
</table>

In [ ]:
import pandas as pd
from pathlib import Path
from scipy import stats
import subprocess
import sqlite3
from tqdm.notebook import tqdm, trange

import plotly
import plotly.express as px
pd.options.plotting.backend = 'plotly'

In [ ]:
PATH = Path('/kaggle/input/riiid-test-answer-prediction')
DB = 'riiid.db'

dtypes_orig = {
    'lectures': {
        'lecture_id': 'uint16',
        'tag': 'uint8',
        'part': 'uint8',
        'type_of': 'str',
    },
    'questions': {
        'question_id': 'uint16',
        'bundle_id': 'uint16',
        'correct_answer': 'uint8',
        'part': 'uint8',
        'tags': 'str',
        
    },
    'train': {
        'row_id': 'int64',
        'timestamp': 'int64',
        'user_id': 'int32',
        'content_id': 'int16',
        'content_type_id': 'int8',
        'task_container_id': 'int16',
        'user_answer': 'int8',
        'answered_correctly': 'int8',
        'prior_question_elapsed_time': 'float32', 
        'prior_question_had_explanation': 'bool'
    }
}

type_map = {
    'int64': 'INTEGER',
    'int32': 'INTEGER',
    'int16': 'INTEGER',
    'int8': 'INTEGER',
    'uint8': 'INTEGER',
    'uint16': 'INTEGER',
    'str': 'TEXT',
    'bool': 'INTEGER',
    'float32': 'REAL'
}

In [ ]:
conn = sqlite3.connect(DB)
c = conn.cursor()

In [ ]:
def sql_create_table(table_id):
    primary_keys = {'lectures': 'lecture_id',  'questions': 'question_id', 'train': 'row_id'}
    primary_key = primary_keys[table_id]

    columns = [f'{primary_key} INTEGER PRIMARY KEY']
    
    columns = columns + [f'{c} {type_map[t]}' for c,t in dtypes_orig[table_id].items()
                if c != primary_keys[table_id]]
        
    columns = (', ').join(columns)
    
    return f"""
        DROP TABLE IF EXISTS {table_id};
        
        CREATE TABLE {table_id}
            ({columns});
        """

def sql_update_fold_seq_start(fold, beg_pct=0.1, late_pct=0.6):
    return f"""
        UPDATE folds
        SET seq_start = 'beg',
            task_container_id_min = 0
        WHERE fold = {fold} AND ABS(RANDOM() % 100) < {int(beg_pct * 100)};
        
        UPDATE folds
        SET seq_start = 'late',
            task_container_id_min = task_container_id_max * 0.5 + (task_container_id_max * 0.5) * ABS(RANDOM() % 100) / 100
        WHERE fold = {fold}
            AND task_container_id_min is NULL
            AND ABS(RANDOM() % 100) < {int(100 * (late_pct / (1 - beg_pct)))};
            
        UPDATE folds
        SET seq_start = 'early',
            task_container_id_min = task_container_id_max * 0.5 * ABS(RANDOM() % 100) / 100
        WHERE fold = {fold}
            AND task_container_id_min is NULL;
    """

## Load Tables

In [ ]:
for table_id in dtypes_orig:
    c.executescript(sql_create_table(table_id))
    conn.commit()

In [ ]:
%%time
for t in tqdm(dtypes_orig):
    result = subprocess.run(['sqlite3', DB, '.mode csv',
                             f'.import {(PATH/t).absolute()}.csv {t}'],
                            capture_output=True, encoding='utf-8')
    
    n_records = c.execute(f'select count(*) from {t}').fetchone()[0]
    print(f'Loaded {n_records:0,d} to {t} table.')
    
# datatype mismatch is the header column failing
# --skip n not implemented in sqlite3 until version 3.32

### Clean Up a Few Columns

In [ ]:
c.execute("""
    UPDATE train
    set prior_question_elapsed_time = "0"
    WHERE prior_question_elapsed_time = "";
""").fetchone()

In [ ]:
c.executescript("""
    ALTER TABLE train
        RENAME prior_question_had_explanation TO prior_question_had_explanation_string;
    
    ALTER TABLE train
        ADD COLUMN prior_question_had_explanation INTEGER;
""").fetchone()

In [ ]:
%%time
c.execute("""
    UPDATE train
    SET prior_question_had_explanation = (
        SELECT t.prior_question_had_explanation_string = "True"
        FROM train t
        WHERE train.row_id = t.row_id
    )
""").fetchone()

In [ ]:
pd.read_sql('select * from train limit 10', conn)

In [ ]:
%%time
c.execute("""
    CREATE INDEX train_user_id_task_container_id_index ON train (user_id, task_container_id);
    """).fetchone()

In [ ]:
%%time
c.execute("""        
    UPDATE train
    SET task_container_id = (
        SELECT DENSE_RANK()
        OVER (
          PARTITION BY user_id
          ORDER BY timestamp
        ) - 1
        FROM train t
        WHERE train.row_id = t.row_id
    )
""").fetchone()
conn.commit()

In [ ]:
db_size = pd.read_sql('SELECT page_count * page_size as size FROM pragma_page_count(), pragma_page_size()', conn)['size'][0]
print(f'Total size of database is: {db_size/1e9:0.3f} GB')

## Difference in Early and Late Performance

This illustrates nicely the difference in peformance between users' early and late performance. This takes the difference in the percentage of questions answered correctly in the first ten task containers and compares it to the percentage of questions answered correctly in the last ten task containers. The differences are then sorted starting with those that had the biggest decrease in performance and ending with those that had the biggest increase. If the performance was the same, there would be the same amount of purple on the left as there is red on the right, but you can clearly see that there are a lot more users with increases in performance.

I ran this for 100,000 users and the difference was as a staggering ten points, from an average of 51% in the first ten task containers, to an average of 61% in the last ten. The p-value on the t-statistic is basically zero. As an aside, it is interesting to note that the average over the last 10 task containers is actually lower than the overall average of approximately 65%, perhaps supporting the hypothesis that the later questions are more difficult than the earlier ones.

In [ ]:
%%time
df_users = pd.read_sql("""
    SELECT t.user_id,
        MAX(c2.task_container_id_total) task_container_id_total,
        MAX(c2.row_id_count_total) row_id_count_total,
        MAX(c2.answered_correctly_total) answered_correctly_total,
        MAX(c2.task_container_id_early) task_container_id_early,
        MAX(c2.row_id_count_early) row_id_count_early,
        MAX(c2.ansered_correctly_early) answered_correctly_early,
        MIN(t.task_container_id) task_container_id_late,
        COUNT(t.row_id) row_id_count_late,
        SUM(t.answered_correctly) answered_correctly_late,
        ROUND(MAX(c2.ansered_correctly_early) * 100 / MAX(c2.row_id_count_early), 2) answered_correctly_early_pct,
        ROUND(SUM(t.answered_correctly) * 100 / COUNT(t.row_id), 2) answered_correctly_late_pct,
        ROUND(MAX(c2.answered_correctly_total) * 100 / MAX(c2.row_id_count_total), 2) answered_correctly_total_pct,
        ROUND(MAX(c2.answered_correctly_total) * 100 / MAX(c2.row_id_count_total), 2) - ROUND(MAX(c2.ansered_correctly_early) * 100 / MAX(c2.row_id_count_early), 2) late_early_delta
    FROM (
        SELECT c.user_id,
        MAX(t.task_container_id) task_container_id_early,
        MAX(c.task_container_id) task_container_id_total,
        SUM(t.answered_correctly) ansered_correctly_early,
        MAX(c.answered_correctly) answered_correctly_total,
        MAX(c.row_id_count) row_id_count_total,
        COUNT(t.row_id) row_id_count_early
        FROM (
            SELECT user_id, MAX(task_container_id) task_container_id, SUM(answered_correctly) answered_correctly, COUNT(row_id) row_id_count
            FROM train
            WHERE content_type_id = 0
            GROUP BY user_id
            LIMIT 100000
        ) c
        JOIN train t
        ON t.user_id = c.user_id AND t.content_type_id = 0 AND t.task_container_id <  10 /* c.task_container_id * 0.1 */
        GROUP BY c.user_id
        ORDER BY c.user_id
    ) c2
    JOIN train t
    ON t.user_id = c2.user_id AND t.content_type_id = 0 AND t.task_container_id > c2.task_container_id_total - 10 /* * 0.1 */
    WHERE c2.task_container_id_total > 30
    GROUP BY c2.user_id
    ORDER BY c2.user_id

    """, conn)

In [ ]:
df_users['positive_delta'] = df_users.late_early_delta > 0
fig = (df_users[['user_id', 'late_early_delta', 'positive_delta']]
       .iloc[600:800].sort_values('late_early_delta').set_index('user_id')
       .plot(kind='bar', y='late_early_delta', color='positive_delta',
             title='Difference in Percent Answered Correctly of - First 10 Task Container Ids vs. Last 10'))
fig.update_xaxes(type='category')
fig.show()

In [ ]:
stats.ttest_ind(df_users.answered_correctly_early_pct, df_users.answered_correctly_late_pct)

## Construct Folds

In [ ]:
c.executescript("""
    DROP TABLE IF EXISTS folds;

    CREATE TABLE folds (
        user_id INTEGER PRIMARY KEY,
        fold INTEGER,
        seq_start TEXT,
        task_container_id_min INTEGER,
        task_container_id_max INTEGER);
""").fetchone()

In [ ]:
%%time
n_folds = 40
c.executescript(f"""
    INSERT INTO folds (user_id, task_container_id_max)
    SELECT user_id, MAX(task_container_id) task_container_id_max
    FROM train
    GROUP BY user_id
    ORDER BY user_id;
    
    UPDATE folds
    SET fold = ABS(RANDOM() % {n_folds});
""").fetchone()

In [ ]:
%%time
c.execute("""
    CREATE INDEX folds_user_id_index ON folds (user_id);
    """).fetchone()

In [ ]:
def get_df_folds():

    return pd.read_sql("""
        SELECT fold, seq_start, COUNT(user_id) user_count,
        AVG(task_container_id_min) task_container_id_min_avg,
        AVG(task_container_id_max) task_container_id_max_avg
        FROM folds
        GROUP BY fold, seq_start;
    """, conn)

df_folds = get_df_folds()

## Review Folds

The charts below show that folds have basically the same number of users and records in them.

In [ ]:
df_folds.groupby('fold').sum().plot(kind='bar', y='user_count',
                                title='User Count by Fold')

In [ ]:
df_folds.groupby('fold').sum().plot(kind='bar', y='task_container_id_max_avg',
                                title='Average Task Containers by Fold')

## Set Minimum Task Container Id to Include

Here we set the minimum task container id that will be included for users in each of the three groups. Note that the mix of records shifts towards the earlier groups since by design we are including smaller percentages of the later groups.

<table>
    <tr>
        <th>Group</th>
        <th>Users</th>
        <th>Records</th>
        <th>Description</th>
    </tr>
    <tr>
        <td>Beginning</td>
        <td>10%</td>
        <td>~20%</td>
        <td>
            <ul>
                <li>Includes sequences starting with users' initial interaction</li>
                <li>Minimum task container id = 0</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td>Late</td>
        <td>60%</td>
        <td>~30%</td>
        <td>
            <ul>
                <li>Includes sequences starting in the later half of users' task containers</li>
                <li>Minimum included task container id determined by selecting randomly from second half of users' task containers</li>
                <li>Effectively averages to including 25% of users' records</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td>Early</td>
        <td>30%</td>
        <td>~50%</td>
        <td>
            <ul>
                <li>Includes sequences starting in the first half of users' task containers</li>
                <li>Minimum included task container id determined by selecting randomly from first half of users' task containers</li>
                <li>Effectively averages to including 75% of users' records</li>
            </ul>
        </td>
    </tr>
</table>

In [ ]:
%%time
c.execute("""
    UPDATE folds
    SET task_container_id_min = NULL,
        seq_start = NULL;
""").fetchone()

for f in trange(n_folds):
    c.executescript(sql_update_fold_seq_start(f)).fetchone()

In [ ]:
df_folds_seq = (get_df_folds().groupby(['fold', 'seq_start']).max()
                   .unstack('seq_start').reset_index().set_index('fold'))

This chart shows that the desired mix of beginning, early and late **users** achieved.

In [ ]:
(df_folds_seq.user_count
 .plot(kind='bar', title='Composition of Folds by Sequence Start - Users'))

This chart shows that each of the three sets of user sequences were basically distributed similarly with respect to task containers before determining where in those sequences to start including them.

In [ ]:
(df_folds_seq.task_container_id_max_avg
 .plot(title='Average Max Task Container Id by Fold'))

And this chart shows where in the users' sequences of task containers they started getting included in the folds. The beginning group obviously starts at zero, the late group starts at an average of approximately 100 and and the early group starts at an average of approximately 50. The late group starts at approximately 75% of the way through the sequences and the early group starts at approximately 25% of the way through the sequences.

In [ ]:
(df_folds_seq.task_container_id_min_avg
 .plot(title='Average Starting Task Container Id by Fold'))

## Add Fold Column to Train Table

In [ ]:
c.executescript("""
    ALTER TABLE train
        ADD COLUMN fold INTEGER;
""").fetchone()

In [ ]:
if False:
    c.execute("""
        UPDATE train
        SET fold = NULL;
    """).fetchone()

In [ ]:
%%time
c.executescript("""
    UPDATE train
    SET fold = (
        SELECT folds.fold
        FROM folds
        WHERE
            train.user_id = folds.user_id
            AND train.task_container_id >= folds.task_container_id_min
    );
""").fetchone()

In [ ]:
%%time
c.execute("""
    CREATE INDEX train_fold_index ON train (fold);
    """).fetchone()
conn.commit()

Here is a two columned dataframe that can be joined with your data to select folds.

In [ ]:
%%time
(pd.read_sql("""
    SELECT row_id, fold
    FROM train
    WHERE fold is not NULL
    ORDER BY row_id
    """, conn)
 .astype({'row_id': 'int64', 'fold': 'int8'})
 .to_pickle('df_folds.pkl'))

## Create Dataframe of Selected Folds

This is how you can query the database to select folds for training and validation.

In [ ]:
%%time

folds_valid = [0]
folds_train = list(range(1,5))
folds = folds_valid + folds_train

df_train = pd.read_sql(f"""
    SELECT t.user_id, t.fold, f.seq_start, t.task_container_id,
        t.row_id, t.answered_correctly
    FROM train t
    JOIN folds f
    ON t.user_id = f.user_id
    WHERE t.fold in ({(', ').join([str(f) for f in folds])})
        AND content_type_id = 0
    """, conn)

The charts above were based on users and task container ids. The charts below are run based on the included records and confirm that they are distrbuted similarly.

In [ ]:
title = 'Record County by Fold'
df_train.fold.value_counts().plot(kind='bar', title=title)

Here we can see that the composition of records has shifted towards the earlier groups since by design we are including fewer of the records from the users in the later groups.

In [ ]:
df_tfg = df_train.groupby(['fold', 'seq_start']).count()['row_id'].unstack('seq_start')
df_ts = df_tfg.sum(axis=1)
(df_tfg.divide(df_ts / 100, axis=0)
 .plot(kind='bar', barmode='stack', title='Composition of Folds by Sequence Start - Records'))

In [ ]:
(df_train.groupby(['fold','user_id'])[['fold','user_id']].head(1)
 .reset_index().groupby('fold').count().user_id
 .plot(kind='bar', title='Count of Users by Fold'))

In [ ]:
df_train_g_fold = df_train[['user_id', 'fold', 'answered_correctly']].groupby('fold').agg({'user_id': 'count', 'answered_correctly': 'sum'})
df_train_g_fold['answered_correctly_avg'] = (df_train_g_fold.answered_correctly * 100000 / df_train_g_fold.user_id) // 100 / 10
fig = df_train_g_fold.reset_index().plot(kind='bar', y='answered_correctly_avg', title='Average Percent Answered Correctly by Fold')
fig.show()

In [ ]:
df_train['valid'] = df_train.fold.isin(folds_valid)

In [ ]:
def show_plot(agg, title, labels):
    g_user_ct = (df_train[['valid', 'row_id', 'seq_start', 'user_id']]
                 .groupby(['valid', 'seq_start', 'user_id']).count().unstack('seq_start').fillna(0))

    bins = [0,10,20,50,100,250,500,1000,2500,5000,20000]
    g_user_ct['bin'] = pd.cut(g_user_ct.row_id.sum(axis=1), bins=bins, duplicates='drop')
    g_counts = g_user_ct.sort_index().groupby(['valid', 'bin']).agg(agg).reset_index()
    g_counts.columns = ['valid', 'bin', 'beg', 'early', 'late']
    g_counts.bin = g_counts.bin.astype(str)

    fig = g_counts.plot(kind='bar', x='bin', y=['beg', 'early', 'late'], facet_col='valid',
             title=title,
             labels=labels)
    
    fig.show()

show_plot('count',
          title='Count of Users by Records per User by Sequence Group',
          labels={'bin': 'Records per User', 'value': 'Count of Users'}
         )

In [ ]:
show_plot('sum',
          title='Count of Records by Records per User by Sequence Group',
          labels={'bin': 'Records per User', 'value': 'Count of Records'}
         )

That's all folks - if you made it here, you are dedicated - thanks for reading!